In [ ]:
import os

import torch
from PIL import Image
from lang_sam import LangSAM
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
model = LangSAM()
text_prompt = "clothing, shoes, pants"
df = pd.read_csv("datasets/query_list.csv")
override_masks = False
dataset_path = "datasets/deep-fashion/In-shop Clothes Retrieval Benchmark/Img/"
for path in tqdm(df["image_name"]):
    mask_path = dataset_path + (".".join(path.split(".")[:-1])) + "_mask_sam.png"
    if override_masks or not os.path.exists(mask_path):
        image_pil = Image.open(dataset_path + path).convert("RGB")
        masks, _, _, _ = model.predict(image_pil, text_prompt, text_threshold=0.4)
        
        # Combine all detected objects into one mask
        all_detections_mask = masks[0]
        for mask in masks:
            all_detections_mask = torch.logical_or(all_detections_mask, mask)
    
        plt.imsave(mask_path, arr=all_detections_mask, cmap="Greys")